 # Create a Twisted Interface with Commensurate Lattices

Use commensurate lattice matching algorithm to create twisted interfaces between two materials.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete (depending on the parameters can take a few min).
1. Scroll down to view results.

## Summary

1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create and visualize the initial materials
1. Generate twisted interfaces with commensurate lattice matching
1. Select the interface with the desired twist angle and visualize it

## Notes

1. We perform commensurate lattice matching to find valid supercells that achieve the desired twist angle.
1. When the matching is finished, interfaces with angles close to the target are presented.
1. The algorithm searches for supercell matrices within specified size limits.
1. For more information, see [Introduction](Introduction.ipynb)


## 1. Prepare the Environment
### 1.1. Set up the notebook
Set the following flags to control the notebook behavior


In [ ]:
# Material selection and basic parameters
FILM_INDEX = 0# Index in the list of materials, to access as materials[FILM_INDEX]
SUBSTRATE_INDEX = None  # Can be None to use same material as film

# Twisted interface parameters
TARGET_TWIST_ANGLE = 17.9  # in degrees
INTERFACE_DISTANCE = 3.0  # in Angstroms
INTERFACE_VACUUM = 20.0  # in Angstroms

# Search algorithm parameters
MAX_REPETITION = 6  # Maximum supercell matrix element value
ANGLE_TOLERANCE = 2.5  # in degrees
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
VISUALIZE_REPETITIONS = [3, 3, 1]

### 1.2. Install packages
Explanation is [here](under_the_hood.ipynb#1.2.-Install-packages).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("")

### 1.3. Load and preview input materials
Explanation is [here](under_the_hood.ipynb#2.-Data-Exchange).

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

 ## 2. Prepare Materials
 ### 2.1. Select and visualize initial materials

In [ ]:
from utils.visualize import visualize_materials

film = materials[FILM_INDEX]
substrate = materials[SUBSTRATE_INDEX] if SUBSTRATE_INDEX is not None else film

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(film, repetitions=VISUALIZE_REPETITIONS)
    if substrate is not film:
        visualize_materials(substrate, repetitions=VISUALIZE_REPETITIONS)

## 3. Generate Twisted Interface
### 3.1. Set up interface configuration and builder


In [ ]:
from mat3ra.made.tools.build.interface.builders import CommensurateLatticeTwistedInterfaceBuilderParameters, \
    CommensurateLatticeTwistedInterfaceBuilder
from mat3ra.made.tools.build.interface.configuration import TwistedInterfaceConfiguration

config = TwistedInterfaceConfiguration(
    film=film,
    substrate=film,
    twist_angle=TARGET_TWIST_ANGLE,
    distance_z=INTERFACE_DISTANCE
)

params = CommensurateLatticeTwistedInterfaceBuilderParameters(
    max_supercell_matrix_int=MAX_REPETITION,
    angle_tolerance=ANGLE_TOLERANCE,
    return_first_match=RETURN_FIRST_MATCH
)

builder = CommensurateLatticeTwistedInterfaceBuilder(build_parameters=params)

### 3.2. Generate and analyze interfaces


In [ ]:
from utils.plot import plot_twisted_interface_solutions

interfaces = builder.get_materials(config)

print(f"\nFound {len(interfaces)} possible interface(s)")
for i, interface in enumerate(interfaces):
    actual_angle = interface.metadata.get("actual_twist_angle", "unknown")
    print(f"\nInterface {i+1}:")
    print(f"Actual twist angle: {actual_angle}°")
    print(f"Number of atoms: {len(interface.basis.elements.ids)}")


if len(interfaces) > 0:
    plot_twisted_interface_solutions(interfaces)

## 4. Preview the selected material
By default, the first interface is selected. You can change the selection by changing the `selected_interface` index.

In [ ]:
selected_interface = interfaces[0]
actual_angle = selected_interface.metadata.get("build").get("configuration").get("actual_twist_angle")
print(f"Target angle: {TARGET_TWIST_ANGLE}°")
print(f"Actual angle: {actual_angle}°")
print(f"Number of atoms: {len(selected_interface.basis.elements.ids)}")

visualize_materials(selected_interface, repetitions=[1, 1, 1])
visualize_materials(selected_interface, repetitions=[1, 1, 1], rotation="-90x")

### 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials

set_materials(selected_interface)